In [ ]:
# ! pip install pyvisa
# ! pip install toml

In [157]:
%load_ext autoreload
%autoreload 2
import os    
import sys    
import time
import pprint
import json
from bs4 import BeautifulSoup
import requests
import json


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# QGIS help page crawler

## How to use?
1. Update the `url` variable using the links in [https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/index.html](https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/index.html), e.g., [https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html](https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html).
2. Run the program. 

**Please check the results manually to ensure they are correct!**

In [324]:
# ! pip install html5lib

In [45]:
import toml
import os
import re 
import io
import requests

In [7]:
##### These urls were processed.
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html"
# there are 156 sections, and 33 sections have h2 tag, while 31 tool sections.

# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/interpolation.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectoranalysis.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorgeneral.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorgeometry.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rasteranalysis.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/plots.html"
# url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasterterrainanalysis.html"


url = "https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectoroverlay.html"

def get_response(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        html_content = response.content
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html5lib')
        # soup = BeautifulSoup(html_content, 'html.parser')

        # Now you can work with 'soup' to extract the relevant sections
        # Example: printing the title of the page
        print("Title: ", soup.title.get_text())

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        
    return soup

soup = get_response(url)

Title:  28.1.23. Vector overlay — QGIS Documentation  documentation


In [ ]:
urls[-1]

In [ ]:
raw_sections = soup.find_all('section')
len(raw_sections)

In [ ]:
# for idx, section in enumerate(sections):
    # print(idx)
    # print(section)

In [ ]:
def get_sections(soup):
    raw_sections = soup.find_all('section')
    len(raw_sections)
    cnt = 0
    sections = []
    for idx, section in enumerate(raw_sections):

        section_id = section.get('id')  # Extract the section ID
        tag = section.find('h2')  # Try to find an <h3> tag in the section
        # print(tag)
        if tag:
            cnt += 1
            print(idx, cnt, f"Section with ID '{section_id}' contains an <h3> tag: '{tag.get_text()}'")
            # cnt += 1
            sections.append(section)
        else:
            # print(idx, f"Section with ID '{section_id}' does not contain an <h3> tag.")
            pass
    # len(sections)  
    return sections

sections = get_sections(soup)

In [ ]:
def get_section_by_heading(section, level, heading):        
    tags = section.find_all(level)
    for tag in tags:
        if heading in tag.get_text():
            section_tag = tag.find_parent('section')
            # print(section_tag.prettify())  # Print the entire section for visualization
            return section_tag
    return None
    
def extract_table(section):
    # print("section:", section)
    table_data = []
    tables = section.find_all('table')
    # print("table:", table)
    for table in tables:
        if table:
            headers = [th.get_text(strip=True) for th in table.find_all('th')]
            rows = table.find_all('tr')[1:]  # Skip header row
            for row in rows:
                cols = row.find_all('td')
                # row_data = {headers[i]: cols[i].get_text(strip=True) for i in range(len(headers))}
                row_data = {headers[i]: cols[i].get_text(separator=' ', strip=True) for i in range(len(headers))}
                table_data.append(row_data)
    return table_data
    
# Helper function to normalize whitespace in a string
def normalize_whitespace(text):
    # Replace multiple spaces and newlines with a single space
    return re.sub(r'\s+', ' ', text).strip()
    
# Function to extract paragraphs before a section with a specific id and ensure spaces around <a> and <code> elements
def extract_paragraphs_before(section, stop_section_id):
    paragraphs = []
    
    for element in section.find_all(['p', 'h3'], recursive=False):
        # Stop if we encounter the sub-section (like "Parameters")
        if element.name == 'h3' and stop_section_id in element.get('id', ''):
            break
        if element.name == 'p':
            # Rebuild the paragraph's text, ensuring spaces around <a> and <code> tags
            paragraph_text = []
            for content in element.children:
                
                if content.name in ['a', 'code']:
                    paragraph_text.append(f" {content.get_text(strip=True)} ")
                    # print(content)
                else:
                    paragraph_text.append(content if isinstance(content, str) else content.get_text(strip=True))
                    # print(paragraph_text[-1])
            
            paragraphs.append(normalize_whitespace(''.join(paragraph_text)))
    
    return paragraphs
    
def make_parameters_for_TOML(tool_info):
    lines = []
    # for parameter in tool_info['basic_parameters']:
    #     line = f"{parameter['Name']}: {parameter['Description'].replace('\n', '')}. Type: {parameter['Type']}"
    #     lines.append(line)
        
    # for parameter in tool_info['advanced_parameters']:
    #     line = f"{parameter['Name']}: {parameter['Description'].replace('\n', '')}. Type: {parameter['Type']}"
    #     lines.append(line)

    for parameter in tool_info['parameters']:
        name = parameter['Name']
        label = parameter['Label'].replace('\n', '')
        description = parameter['Description'].replace('\n', '')
        param_type = parameter['Type']

        line = f"{name}: {label}. {description}. Type: {param_type}"
        lines.append(line)
    
    para_str = "\n".join(lines)
    return para_str

def make_outputs_for_TOML(tool_info):
    lines = []
    for output in tool_info['outputs']:
        try:
            name = output['Name']
            label = output['Label'].replace('\n', '')
            description = output['Description'].replace('\n', '')
            output_type = output['Type']

            line = f"{name}: {label}. {description}. Type: {output_type}"
            lines.append(line)
        except Exception as e:
            print("Error in make_outputs_for_TOML():", e)
    
    para_str = "\n".join(lines)
    return para_str
def make_TOML_file(tool_info, fname):
    tool_toml = {}
    tool_toml['tool_ID'] = tool_info['algorithm_id']
    tool_toml['tool_name'] = tool_info['tool_name']    
    tool_toml['brief_description'] = tool_info['brief_description']
    tool_toml['full_description'] = '\n'.join(tool_info['paragraphs'])
    tool_toml['parameters'] = make_parameters_for_TOML(tool_info)
    tool_toml['outputs'] = make_outputs_for_TOML(tool_info)
    tool_toml['code_example'] = ""
    with open(fname, 'w', encoding='utf-8') as f:  # charmap' codec can't encode character
        toml_str = toml.dump(tool_toml, f)
        
    # toml_str = toml.dumps(tool_toml)
    return toml_str
    
def extract_tool_info(section):

    tool_info = {}
    section_id = section.get('id')
    # print("section_id:", section_id)
    
    h2 = section.find("h2")
    
    if h2:  # Check if an h2 element was found

        paragraph = section.find("p")
        if paragraph:
            paragraphs = extract_paragraphs_before(section, 'parameters')
 
            if len(paragraphs) > 0:
                tool_info['brief_description'] = paragraphs[0]
                # print("Tool description:", tool_info['brief_description'])  
            else: 
                tool_info['brief_description'] = ""
            tool_info['tool_name'] = h2.get_text(strip=True)[:-1].split(".")[-1] 
            tool_info['paragraphs'] = paragraphs
  
        algorithm_id = None 
        python_code_snippet = None
        
        python_code_section = get_section_by_heading(section, 'h3', 'Python code')
        if python_code_section: 
            algorithm_id = python_code_section.find('code').get_text(strip=True)
            if not algorithm_id:
                return tool_info
            # print("algorithm_id:", algorithm_id) 
            pre_tag = python_code_section.find('pre')
            # print("python_code_section:", python_code_section)
            if pre_tag:
                python_code_snippet = pre_tag.get_text()
            # print("algorithm_id:", algorithm_id) 

        # store algorithm_id
        tool_info['algorithm_id'] = algorithm_id
        tool_info['python_code_snippet'] = python_code_snippet

        paremeters_section = get_section_by_heading(section, 'h3', 'Parameters')
        if paremeters_section:
            # print("paremeters_section:", paremeters_section)
            # basic_parameters_section = section.find('section', id='basic-parameters')   # only for 28.1.15. Raster analysis 
            # advanced_parameters_section = section.find('section', id='advanced-parameters') # only for 28.1.15. Raster analysis
            
            
            # basic_parameters = extract_table(basic_parameters_section) if basic_parameters_section else []  # only for 28.1.15. Raster analysis
            # advanced_parameters = extract_table(advanced_parameters_section) if advanced_parameters_section else []  # only for 28.1.15. Raster analysis
            parameters = extract_table(paremeters_section) if paremeters_section else [] 
            # print("parameters:", parameters)

            # tool_info['basic_parameters'] = basic_parameters   # only for 28.1.15. Raster analysis
            # tool_info['advanced_parameters'] = advanced_parameters   # only for 28.1.15. Raster analysis
            tool_info['parameters'] = parameters
            

        outputs_section = get_section_by_heading(section, 'h3', 'Outputs')
        if outputs_section:
            outputs = extract_table(outputs_section) if outputs_section else []
            tool_info['outputs'] = outputs
            # print("outputs:", outputs)

        # print(tool_info)
        # print("Tool description:", tool_info['brief_description'])  
        return tool_info
    else:
        # print("No h2 found in this section.")
        pass
        
    return tool_info


save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM_Geo\QGIS_plugin\toml'

count = 0

for section in sections[2:]:
    try:
        tool_info = extract_tool_info(section)    
        tool_name = tool_info['algorithm_id'].replace(":", "_")
        
        fname = os.path.join(save_dir, f"{tool_name}.toml")
        toml_str = make_TOML_file(tool_info, fname)
        # print("fname:", fname)
        count += 1

        print(f"{count} tool_name:", tool_name)

    except Exception as e:

        print("error:", e)
        pass

    # if count > 0: 
    #     break

# Download a url list

In [ ]:
urls = ["https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/3dtiles.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/cartography.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/database.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/filetools.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/gps.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/interpolation.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/layertools.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/mesh.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/modelertools.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/networkanalysis.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/plots.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/pointcloudconversion.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/pointclouddatamanagement.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/pointcloudextraction.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rastercreation.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rasterterrainanalysis.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/rastertools.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectoranalysis.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorcreation.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorgeneral.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorgeometry.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectoroverlay.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectorselection.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectortable.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/qgis/vectortiles.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/index.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rasteranalysis.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rasterconversion.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rasterextraction.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rastermiscellaneous.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/rasterprojections.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/vectorconversion.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/vectorgeoprocessing.html",
"https://docs.qgis.org/3.34/en/docs/user_manual/processing_algs/gdal/vectormiscellaneous.html"
       ]

In [ ]:
save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM_Geo\QGIS_plugin\toml'



for idx, url in enumerate(urls):
    print(f"Processing: {idx + 1} / {len(urls)}",  url)
    
    soup = get_response(url)
    sections = get_sections(soup)
    
    count = 0

    for section in sections[2:]:
        try:
            tool_info = extract_tool_info(section)    
            tool_name = tool_info['algorithm_id'].replace(":", "_")

            fname = os.path.join(save_dir, f"{tool_name}.toml")
            toml_str = make_TOML_file(tool_info, fname)
            # print("fname:", fname)
            count += 1

            print(f"{count} tool_name:", tool_name)

        except Exception as e:

            print("error:", e)
            pass
    

In [ ]:
tools = ['Tessellate', 'Aspect', 'Assign projection', 'Buffer vectors', 'Build virtual raster', 'Build virtual vector', 'Clip raster by extent', 'Clip raster by mask layer', 'Clip vector by extent', 'Clip vector by mask layer', 'Color relief', 'Contour', 'Contour Polygons', 'Convert format', 'Dissolve', 'Execute SQL', 'Extract projection', 'Fill nodata', 'gdal2tiles', 'gdal2xyz', 'Raster information', 'Grid (Moving average)', 'Grid (Data metrics)', 'Grid (Inverse distance to a power)', 'Grid (IDW with nearest neighbor searching)', 'Grid (Linear)', 'Grid (Nearest neighbor)', 'Hillshade', 'Export to PostgreSQL (available connections)', 'Export to PostgreSQL (new connection)', 'Merge', 'Near black', 'Offset curve', 'Vector information', 'One side buffer', 'Build overviews (pyramids)', 'Pansharpening', 'PCT to RGB', 'Points along lines', 'Polygonize (raster to vector)', 'Proximity (raster distance)', 'Raster calculator', 'Rasterize (vector to raster)', 'Rasterize (overwrite with attribute)', 'Rasterize (overwrite with fixed value)', 'Rearrange bands', 'Retile', 'RGB to PCT', 'Roughness', 'Sieve', 'Slope', 'Tile index', 'Topographic Position Index (TPI)', 'Translate (convert format)', 'Terrain Ruggedness Index (TRI)', 'Viewshed', 'Warp (reproject)', 'g.extension.list', 'g.extension.manage', 'i.albedo', 'i.aster.toar', 'i.atcorr', 'i.biomass', 'i.cca', 'i.cluster', 'i.colors.enhance', 'i.eb.eta', 'i.eb.evapfr', 'i.eb.hsebal01.coords', 'i.eb.netrad', 'i.eb.soilheatflux', 'i.emissivity', 'i.evapo.mh', 'i.evapo.pm', 'i.evapo.pt', 'i.evapo.time', 'i.fft', 'i.gensig', 'i.gensigset', 'i.group', 'i.his.rgb', 'i.ifft', 'i.image.mosaic', 'i.in.spotvgt', 'i.landsat.acca', 'i.landsat.toar', 'i.maxlik', 'i.modis.qc', 'i.oif', 'i.pansharpen', 'i.pca', 'i.rgb.his', 'i.segment', 'i.smap', 'i.tasscap', 'i.topo.coor.ill', 'i.topo.corr', 'i.vi', 'i.zc', 'm.cogo', 'nviz', 'r.basins.fill', 'r.blend.combine', 'r.blend.rgb', 'r.buffer', 'r.buffer.lowmem', 'r.carve', 'r.category', 'r.category.out', 'r.circle', 'r.clump', 'r.coin', 'r.colors', 'r.colors.out', 'r.colors.stddev', 'r.composite', 'r.contour', 'r.cost', 'r.covar', 'r.cross', 'r.describe', 'r.distance', 'r.drain', 'r.fill.dir', 'r.fill.stats', 'r.fillnulls', 'r.flow', 'r.geomorphon', 'r.grow', 'r.grow.distance', 'r.gwflow', 'r.his', 'r.horizon', 'r.horizon.height', 'r.in.lidar', 'r.in.lidar.info', 'r.info', 'r.kappa', 'r.lake', 'r.latlong', 'r.li.cwed', 'r.li.cwed.ascii', 'r.li.dominance', 'r.li.dominance.ascii', 'r.li.edgedensity', 'r.li.edgedensity.ascii', 'r.li.mpa', 'r.li.mpa.ascii', 'r.li.mps', 'r.li.mps.ascii', 'r.li.padcv', 'r.li.padcv.ascii', 'r.li.padrange', 'r.li.padrange.ascii', 'r.li.padsd', 'r.li.padsd.ascii', 'r.li.patchdensity', 'r.li.patchdensity.ascii', 'r.li.patchnum', 'r.li.patchnum.ascii', 'r.li.pielou', 'r.li.pielou.ascii', 'r.li.renyi', 'r.li.renyi.ascii', 'r.li.richness', 'r.li.richness.ascii', 'r.li.shannon', 'r.li.shannon.ascii', 'r.li.shape', 'r.li.shape.ascii', 'r.li.simpson', 'r.li.simpson.ascii', 'r.mapcalc.simple', 'r.mask.rast', 'r.mask.vect', 'r.mfilter', 'r.mode', 'r.neighbors', 'r.null', 'r.out.ascii', 'r.out.gridatb', 'r.out.mat', 'r.out.mpeg', 'r.out.png', 'r.out.pov', 'r.out.ppm', 'r.out.ppm3', 'r.out.vrml', 'r.out.vtk', 'r.out.xyz', 'r.param.scale', 'r.patch', 'r.path', 'r.path.coordinate.txt', 'r.plane', 'r.profile', 'r.proj', 'r.quant', 'r.quantile', 'r.quantile.plain', 'r.random', 'r.random.cells', 'r.random.surface', 'r.reclass', 'r.reclass.area', 'r.recode', 'r.regression.line', 'r.regression.multi', 'r.relief', 'r.relief.scaling', 'r.report', 'r.resamp.bspline', 'r.resamp.filter', 'r.resamp.interp', 'r.resamp.rst', 'r.resamp.stats', 'r.resample', 'r.rescale', 'r.rescale.eq', 'r.rgb', 'r.ros', 'r.series', 'r.series.accumulate', 'r.series.interp', 'r.shade', 'r.sim.sediment', 'r.sim.water', 'r.slope.aspect', 'r.solute.transport', 'r.spread', 'r.spreadpath', 'r.statistics', 'r.stats', 'r.stats.quantile.out', 'r.stats.quantile.rast', 'r.stats.zonal', 'r.stream.extract', 'r.sun.incidout', 'r.sun.insoltime', 'r.sunhours', 'r.sunmask.datetime', 'r.sunmask.position', 'r.surf.area', 'r.surf.contour', 'r.surf.fractal', 'r.surf.gauss', 'r.surf.idw', 'r.surf.random', 'r.terraflow', 'r.texture', 'r.thin', 'r.tile', 'r.tileset', 'r.to.vect', 'r.topidx', 'r.topmodel', 'r.topmodel.topidxstats', 'r.transect', 'r.univar', 'r.uslek', 'r.usler', 'r.viewshed', 'r.volume', 'r.walk.coords', 'r.walk.points', 'r.walk.rast', 'r.water.outlet', 'r.watershed', 'r.what.color', 'r.what.coords', 'r.what.points', 'v.buffer', 'v.build.check', 'v.build.polylines', 'v.class', 'v.clean', 'v.cluster', 'v.db.select', 'v.decimate', 'v.delaunay', 'v.dissolve', 'v.distance', 'v.drape', 'v.edit', 'v.extract', 'v.extrude', 'v.generalize', 'v.hull', 'v.in.ascii', 'v.in.dxf', 'v.in.e00', 'v.in.geonames', 'v.in.lidar', 'v.in.lines', 'v.in.mapgen', 'v.in.wfs', 'v.info', 'v.kcv', 'v.kernel.rast', 'v.kernel.vector', 'v.lidar.correction', 'v.lidar.edgedetection', 'v.lidar.growing', 'v.mkgrid', 'v.neighbors', 'v.net', 'v.net.alloc', 'v.net.allpairs', 'v.net.bridge', 'v.net.centrality', 'v.net.components', 'v.net.connectivity', 'v.net.distance', 'v.net.flow', 'v.net.iso', 'v.net.nreport', 'v.net.path', 'v.net.report', 'v.net.salesman', 'v.net.spanningtree', 'v.net.steiner', 'v.net.timetable', 'v.net.visibility', 'v.normal', 'v.out.ascii', 'v.out.dxf', 'v.out.postgis', 'v.out.pov', 'v.out.svg', 'v.out.vtk', 'v.outlier', 'v.overlay', 'v.pack', 'v.parallel', 'v.patch', 'v.perturb', 'v.proj', 'v.qcount', 'v.random', 'v.rast.stats', 'v.reclass', 'v.rectify', 'v.report', 'v.sample', 'v.segment', 'v.select', 'v.split', 'v.surf.bspline', 'v.surf.idw', 'v.surf.rst', 'v.surf.rst.cvdev', 'v.to.3d', 'v.to.lines', 'v.to.points', 'v.to.rast', 'v.transform', 'v.type', 'v.univar', 'v.vect.stats', 'v.voronoi', 'v.voronoi.skeleton', 'v.what.rast', 'v.what.vect', 'Add autoincremental field', 'Add field to attributes table', 'Add unique value index field', 'Add X/Y fields to layer', 'Affine transform', 'Aggregate', 'Align rasters', 'Align raster', 'Align points to features', 'Geodesic line split at antimeridian', 'Array of offset (parallel) lines', 'Array of translated features', 'Aspect', 'Assign projection', 'Export atlas layout as image', 'Export atlas layout as PDF (multiple files)', 'Export atlas layout as PDF (single file)', 'Convert B3DM to GLTF', 'Batch Nominatim geocoder', 'Convert spatial bookmarks to layer', 'Boundary', 'Bounding boxes', 'Buffer', 'Variable width buffer (by M value)', 'Calculate expression', 'Overlap analysis', 'Create categorized renderer from styles', 'Cell stack percentile', 'Cell stack percentrank from raster layer', 'Cell stack percent rank from value', 'Cell statistics', 'Centroids', 'Clip', 'Collect geometries', 'Combine style databases', 'Concave hull', 'Conditional branch', 'Convert GPS data', 'Convert GPX feature type', 'Convert to curved geometries', 'Convex hull', 'Count points in polygon', 'Create attribute index', 'Create constant raster layer', 'Create directory', 'Create grid', 'Create points layer from table', 'Create random raster layer (binomial distribution)', 'Create random raster layer (exponential distribution)', 'Create random raster layer (gamma distribution)', 'Create random raster layer (geometric distribution)', 'Create random raster layer (negative binomial distribution)', 'Create random raster layer (normal distribution)', 'Create random raster layer (poisson distribution)', 'Create random raster layer (uniform distribution)', 'Create spatial index', 'DBSCAN clustering', 'Delaunay triangulation', 'Drop field(s)', 'Delete duplicate geometries', 'Delete holes', 'Densify by count', 'Densify by interval', 'Detect dataset changes', 'Difference', 'Dissolve', 'Download GPS data from device', 'Download vector tiles', 'Drop geometries', 'Drop M/Z values', 'DTM filter (slope-based)', 'Export layers to DXF', 'Equal to frequency', 'Explode HStore Field', 'Explode lines', 'Export layer(s) information', 'Export mesh edges', 'Export mesh faces', 'Export mesh on grid', 'Export mesh vertices', 'Export to spreadsheet', 'Extend lines', 'Create layer from extent', 'Extract binary field', 'Extract by attribute', 'Extract by expression', 'Extract/clip by extent', 'Extract by location', 'Extract labels', 'Extract M values', 'Extract specific vertices', 'Extract vertices', 'Extract within distance', 'Extract Z values', 'Field calculator', 'Download file', 'Fill NoData cells', 'Feature filter', 'Filter by geometry type', 'Filter layers by type', 'Filter vertices by M value', 'Filter vertices by Z value', 'Fix geometries', 'Flatten relationship', 'Force right-hand-rule', 'Fuzzify raster (gaussian membership)', 'Fuzzify raster (large membership)', 'Fuzzify raster (linear membership)', 'Fuzzify raster (near membership)', 'Fuzzify raster (power membership)', 'Fuzzify raster (small membership)', 'Generate points (pixel centroids) inside polygons', 'Geometry by expression', 'Convert GLTF to vector features', 'Greater than frequency', 'Highest position in raster stack', 'Hillshade', 'Join by lines (hub lines)', 'Export to PostgreSQL', 'Import geotagged photos', 'Interpolate point on line', 'Intersection', 'Join attributes by location', 'Join attributes by field value', 'Join attributes by location (summary)', 'Join attributes by nearest', 'Keep N biggest parts', 'K-means clustering', 'Convert layer to spatial bookmarks', 'Less than frequency', 'Line density', 'Line intersections', 'Line substring', 'Load layer into project', 'Lowest position in raster stack', 'Mean coordinate(s)', 'Merge lines', 'Merge vector layers', 'Export contours', 'Export cross section dataset values on lines from mesh', 'Export time series values from points of a mesh dataset', 'Rasterize mesh dataset', 'Minimum enclosing circles', 'Raster calculator', 'Raster calculator (virtual)', 'Difference (multiple)', 'Intersection (multiple)', 'Multipart to singleparts', 'Multi-ring buffer (constant distance)', 'Union (multiple)', 'Nearest neighbour analysis', 'Offset lines', 'Order by expression', 'Oriented minimum bounding box', 'Orthogonalize', 'Package layers', 'Raster pixels to points', 'Raster pixels to polygons', 'Point on surface', 'Points along geometry', 'Points to path', 'Create layer from point', 'Pole of inaccessibility', 'Extract layer extent', 'Polygonize', 'Polygons to lines', 'PostgreSQL execute SQL', 'Print layout map extent to layer', 'Export print layout as image', 'Export print layout as PDF', 'Project points (Cartesian)', 'Promote to multipart', 'Raise exception', 'Raise message', 'Raise warning', 'Random extract', 'Random points in extent', 'Random points in polygons', 'Random points on lines', 'Raster boolean AND', 'Raster calculator', 'Convert map to raster', 'Raster layer properties', 'Raster layer statistics', 'Raster layer unique values report', 'Raster layer zonal statistics', 'Raster boolean OR', 'Sample raster values', 'Raster surface volume', 'Reclassify by layer', 'Reclassify by table', 'Rectangles, ovals, diamonds', 'Refactor fields', 'Delete duplicates by attribute', 'Remove duplicate vertices', 'Remove null geometries', 'Rename layer', 'Rename field', 'Repair Shapefile', 'Reproject layer', 'Rescale raster', 'Retain fields', 'Reverse line direction', 'Rotate', 'Roundness', 'Round raster', 'Ruggedness index', 'Save vector features to file', 'Save log to file', 'Extract selected features', 'Segmentize by maximum angle', 'Segmentize by maximum distance', 'Select by location', 'Select within distance', 'Service area (from layer)', 'Service area (from point)', 'Set layer encoding', 'Set layer style', 'Set M value from raster', 'Set M value', 'Set project variable', 'Drape (set Z value from raster)', 'Set Z value', 'Shortest line between features', 'Shortest path (layer to point)', 'Shortest path (point to layer)', 'Shortest path (point to point)', 'Extract Shapefile encoding', 'Simplify', 'Single sided buffer', 'Slope', 'Smooth', 'Snap geometries to layer', 'Snap points to grid', 'SpatiaLite execute SQL', 'SpatiaLite execute SQL (registered DB)', 'Split features by character', 'Split lines by maximum length', 'Split vector layer', 'Split with lines', 'ST-DBSCAN clustering', 'String concatenation', 'Create style database from project', 'Subdivide', 'Sum line lengths', 'Swap X and Y coordinates', 'Symmetrical difference', 'Tapered buffers', 'Generate XYZ tiles (Directory)', 'Generate XYZ tiles (MBTiles)', 'TIN Mesh Creation', 'Transect', 'Transfer annotations from main layer', 'Translate', 'Truncate table', 'Union', 'Upload GPS data to device', 'Raster calculator (virtual)', 'Voronoi polygons', 'Create wedge buffers', 'Write Vector Tiles (MBTiles)', 'Write Vector Tiles (XYZ)', 'Zonal histogram', 'Zonal statistics (in place)', 'Zonal statistics', 'Assign projection', 'Boundary', 'Clip', 'Convert format', 'Create COPC', 'Density', 'Export to raster', 'Export to raster (using triangulation)', 'Export to vector', 'Filter', 'Information', 'Merge', 'Reproject', 'Thin (by skipping points)', 'Thin (by sampling radius)', 'Tile', 'Build virtual point cloud (VPC)', 'Advanced Python field calculator', 'Bar plot', 'Basic statistics for fields', 'Box plot', 'Check validity', 'Climb along line', 'Convert geometry type', 'Define Shapefile projection', 'Distance matrix', 'Distance to nearest hub (line to hub)', 'Distance to nearest hub (points)', 'Eliminate selected polygons', 'Execute SQL', 'Add geometry attributes', 'Find projection', 'Generate points (pixel centroids) along line', 'Heatmap (Kernel Density Estimation)', 'Hypsometric curves', 'IDW interpolation', 'Export to SpatiaLite', 'Concave hull (k-nearest neighbor)', 'Lines to polygons', 'List unique values', 'Mean and standard deviation plot', 'Minimum bounding geometry', 'Points displacement', 'Polar plot', 'PostgreSQL execute and load SQL', 'Random extract within subsets', 'Random points along line', 'Random points in layer bounds', 'Random points inside polygons', 'Random selection', 'Random selection within subsets', 'Raster calculator', 'Raster layer histogram', 'Rectangles, ovals, diamonds (variable)', 'Regular points', 'Relief', 'Vector layer scatterplot 3D', 'Select by attribute', 'Select by expression', 'Set style for raster layer', 'Set style for vector layer', 'Statistics by categories', 'Text to float', 'TIN interpolation', 'Topological coloring', 'Variable distance buffer', 'Vector layer histogram', 'Vector layer scatterplot']
len(tools)

In [ ]:
GRASS_tools =["g.extension.list", "g.extension.manage", "i.albedo", "i.aster.toar", "i.atcorr", "i.biomass", "i.cca", "i.cluster", "i.colors.enhance", "i.eb.eta", "i.eb.evapfr", "i.eb.hsebal01.coords", "i.eb.netrad", "i.eb.soilheatflux", "i.emissivity", "i.evapo.mh", "i.evapo.pm", "i.evapo.pt", "i.evapo.time", "i.fft", "i.gensig", "i.gensigset", "i.group", "i.his.rgb", "i.ifft", "i.image.mosaic", "i.in.spotvgt", "i.landsat.acca", "i.landsat.toar", "i.maxlik", "i.modis.qc", "i.oif", "i.pansharpen", "i.pca", "i.rgb.his", "i.segment", "i.smap", "i.tasscap", "i.topo.coor.ill", "i.topo.corr", "i.vi", "i.zc", "m.cogo", "nviz", "r.basins.fill", "r.blend.combine", "r.blend.rgb", "r.buffer", "r.buffer.lowmem", "r.carve", "r.category", "r.category.out", "r.circle", "r.clump", "r.coin", "r.colors", "r.colors.out", "r.colors.stddev", "r.composite", "r.contour", "r.cost", "r.covar", "r.cross", "r.describe", "r.distance", "r.drain", "r.fill.dir", "r.fill.stats", "r.fillnulls", "r.flow", "r.geomorphon", "r.grow", "r.grow.distance", "r.gwflow", "r.his", "r.horizon", "r.horizon.height", "r.in.lidar", "r.in.lidar.info", "r.info", "r.kappa", "r.lake", "r.latlong", "r.li.cwed", "r.li.cwed.ascii", "r.li.dominance", "r.li.dominance.ascii", "r.li.edgedensity", "r.li.edgedensity.ascii", "r.li.mpa", "r.li.mpa.ascii", "r.li.mps", "r.li.mps.ascii", "r.li.padcv", "r.li.padcv.ascii", "r.li.padrange", "r.li.padrange.ascii", "r.li.padsd", "r.li.padsd.ascii", "r.li.patchdensity", "r.li.patchdensity.ascii", "r.li.patchnum", "r.li.patchnum.ascii", "r.li.pielou", "r.li.pielou.ascii", "r.li.renyi", "r.li.renyi.ascii", "r.li.richness", "r.li.richness.ascii", "r.li.shannon", "r.li.shannon.ascii", "r.li.shape", "r.li.shape.ascii", "r.li.simpson", "r.li.simpson.ascii", "r.mapcalc.simple", "r.mask.rast", "r.mask.vect", "r.mfilter", "r.mode", "r.neighbors", "r.null", "r.out.ascii", "r.out.gridatb", "r.out.mat", "r.out.mpeg", "r.out.png", "r.out.pov", "r.out.ppm", "r.out.ppm3", "r.out.vrml", "r.out.vtk", "r.out.xyz", "r.param.scale", "r.patch", "r.path", "r.path.coordinate.txt", "r.plane", "r.profile", "r.proj", "r.quant", "r.quantile", "r.quantile.plain", "r.random", "r.random.cells", "r.random.surface", "r.reclass", "r.reclass.area", "r.recode", "r.regression.line", "r.regression.multi", "r.relief", "r.relief.scaling", "r.report", "r.resamp.bspline", "r.resamp.filter", "r.resamp.interp", "r.resamp.rst", "r.resamp.stats", "r.resample", "r.rescale", "r.rescale.eq", "r.rgb", "r.ros", "r.series", "r.series.accumulate", "r.series.interp", "r.shade", "r.sim.sediment", "r.sim.water", "r.slope.aspect", "r.solute.transport", "r.spread", "r.spreadpath", "r.statistics", "r.stats", "r.stats.quantile.out", "r.stats.quantile.rast", "r.stats.zonal", "r.stream.extract", "r.sun.incidout", "r.sun.insoltime", "r.sunhours", "r.sunmask.datetime", "r.sunmask.position", "r.surf.area", "r.surf.contour", "r.surf.fractal", "r.surf.gauss", "r.surf.idw", "r.surf.random", "r.terraflow", "r.texture", "r.thin", "r.tile", "r.tileset", "r.to.vect", "r.topidx", "r.topmodel", "r.topmodel.topidxstats", "r.transect", "r.univar", "r.uslek", "r.usler", "r.viewshed", "r.volume", "r.walk.coords", "r.walk.points", "r.walk.rast", "r.water.outlet", "r.watershed", "r.what.color", "r.what.coords", "r.what.points", "v.buffer", "v.build.check", "v.build.polylines", "v.class", "v.clean", "v.cluster", "v.db.select", "v.decimate", "v.delaunay", "v.dissolve", "v.distance", "v.drape", "v.edit", "v.extract", "v.extrude", "v.generalize", "v.hull", "v.in.ascii", "v.in.dxf", "v.in.e00", "v.in.geonames", "v.in.lidar", "v.in.lines", "v.in.mapgen", "v.in.wfs", "v.info", "v.kcv", "v.kernel.rast", "v.kernel.vector", "v.lidar.correction", "v.lidar.edgedetection", "v.lidar.growing", "v.mkgrid", "v.neighbors", "v.net", "v.net.alloc", "v.net.allpairs", "v.net.bridge", "v.net.centrality", "v.net.components", "v.net.connectivity", "v.net.distance", "v.net.flow", "v.net.iso", "v.net.nreport", "v.net.path", "v.net.report", "v.net.salesman", "v.net.spanningtree", "v.net.steiner", "v.net.timetable", "v.net.visibility", "v.normal", "v.out.ascii", "v.out.dxf", "v.out.postgis", "v.out.pov", "v.out.svg", "v.out.vtk", "v.outlier", "v.overlay", "v.pack", "v.parallel", "v.patch", "v.perturb", "v.proj", "v.qcount", "v.random", "v.rast.stats", "v.reclass", "v.rectify", "v.report", "v.sample", "v.segment", "v.select", "v.split", "v.surf.bspline", "v.surf.idw", "v.surf.rst", "v.surf.rst.cvdev", "v.to.3d", "v.to.lines", "v.to.points", "v.to.rast", "v.transform", "v.type", "v.univar", "v.vect.stats", "v.voronoi", "v.voronoi.skeleton", "v.what.rast", "v.what.vect"]

In [ ]:
for t in GRASS_tools:
    print(t)

# Grass Tools

In [ ]:
GRASS_tools =["g.extension.list", "g.extension.manage", "i.albedo", "i.aster.toar", "i.atcorr", "i.biomass", "i.cca", "i.cluster", "i.colors.enhance", "i.eb.eta", "i.eb.evapfr", "i.eb.hsebal01.coords", "i.eb.netrad", "i.eb.soilheatflux", "i.emissivity", "i.evapo.mh", "i.evapo.pm", "i.evapo.pt", "i.evapo.time", "i.fft", "i.gensig", "i.gensigset", "i.group", "i.his.rgb", "i.ifft", "i.image.mosaic", "i.in.spotvgt", "i.landsat.acca", "i.landsat.toar", "i.maxlik", "i.modis.qc", "i.oif", "i.pansharpen", "i.pca", "i.rgb.his", "i.segment", "i.smap", "i.tasscap", "i.topo.coor.ill", "i.topo.corr", "i.vi", "i.zc", "m.cogo", "nviz", "r.basins.fill", "r.blend.combine", "r.blend.rgb", "r.buffer", "r.buffer.lowmem", "r.carve", "r.category", "r.category.out", "r.circle", "r.clump", "r.coin", "r.colors", "r.colors.out", "r.colors.stddev", "r.composite", "r.contour", "r.cost", "r.covar", "r.cross", "r.describe", "r.distance", "r.drain", "r.fill.dir", "r.fill.stats", "r.fillnulls", "r.flow", "r.geomorphon", "r.grow", "r.grow.distance", "r.gwflow", "r.his", "r.horizon", "r.horizon.height", "r.in.lidar", "r.in.lidar.info", "r.info", "r.kappa", "r.lake", "r.latlong", "r.li.cwed", "r.li.cwed.ascii", "r.li.dominance", "r.li.dominance.ascii", "r.li.edgedensity", "r.li.edgedensity.ascii", "r.li.mpa", "r.li.mpa.ascii", "r.li.mps", "r.li.mps.ascii", "r.li.padcv", "r.li.padcv.ascii", "r.li.padrange", "r.li.padrange.ascii", "r.li.padsd", "r.li.padsd.ascii", "r.li.patchdensity", "r.li.patchdensity.ascii", "r.li.patchnum", "r.li.patchnum.ascii", "r.li.pielou", "r.li.pielou.ascii", "r.li.renyi", "r.li.renyi.ascii", "r.li.richness", "r.li.richness.ascii", "r.li.shannon", "r.li.shannon.ascii", "r.li.shape", "r.li.shape.ascii", "r.li.simpson", "r.li.simpson.ascii", "r.mapcalc.simple", "r.mask.rast", "r.mask.vect", "r.mfilter", "r.mode", "r.neighbors", "r.null", "r.out.ascii", "r.out.gridatb", "r.out.mat", "r.out.mpeg", "r.out.png", "r.out.pov", "r.out.ppm", "r.out.ppm3", "r.out.vrml", "r.out.vtk", "r.out.xyz", "r.param.scale", "r.patch", "r.path", "r.path.coordinate.txt", "r.plane", "r.profile", "r.proj", "r.quant", "r.quantile", "r.quantile.plain", "r.random", "r.random.cells", "r.random.surface", "r.reclass", "r.reclass.area", "r.recode", "r.regression.line", "r.regression.multi", "r.relief", "r.relief.scaling", "r.report", "r.resamp.bspline", "r.resamp.filter", "r.resamp.interp", "r.resamp.rst", "r.resamp.stats", "r.resample", "r.rescale", "r.rescale.eq", "r.rgb", "r.ros", "r.series", "r.series.accumulate", "r.series.interp", "r.shade", "r.sim.sediment", "r.sim.water", "r.slope.aspect", "r.solute.transport", "r.spread", "r.spreadpath", "r.statistics", "r.stats", "r.stats.quantile.out", "r.stats.quantile.rast", "r.stats.zonal", "r.stream.extract", "r.sun.incidout", "r.sun.insoltime", "r.sunhours", "r.sunmask.datetime", "r.sunmask.position", "r.surf.area", "r.surf.contour", "r.surf.fractal", "r.surf.gauss", "r.surf.idw", "r.surf.random", "r.terraflow", "r.texture", "r.thin", "r.tile", "r.tileset", "r.to.vect", "r.topidx", "r.topmodel", "r.topmodel.topidxstats", "r.transect", "r.univar", "r.uslek", "r.usler", "r.viewshed", "r.volume", "r.walk.coords", "r.walk.points", "r.walk.rast", "r.water.outlet", "r.watershed", "r.what.color", "r.what.coords", "r.what.points", "v.buffer", "v.build.check", "v.build.polylines", "v.class", "v.clean", "v.cluster", "v.db.select", "v.decimate", "v.delaunay", "v.dissolve", "v.distance", "v.drape", "v.edit", "v.extract", "v.extrude", "v.generalize", "v.hull", "v.in.ascii", "v.in.dxf", "v.in.e00", "v.in.geonames", "v.in.lidar", "v.in.lines", "v.in.mapgen", "v.in.wfs", "v.info", "v.kcv", "v.kernel.rast", "v.kernel.vector", "v.lidar.correction", "v.lidar.edgedetection", "v.lidar.growing", "v.mkgrid", "v.neighbors", "v.net", "v.net.alloc", "v.net.allpairs", "v.net.bridge", "v.net.centrality", "v.net.components", "v.net.connectivity", "v.net.distance", "v.net.flow", "v.net.iso", "v.net.nreport", "v.net.path", "v.net.report", "v.net.salesman", "v.net.spanningtree", "v.net.steiner", "v.net.timetable", "v.net.visibility", "v.normal", "v.out.ascii", "v.out.dxf", "v.out.postgis", "v.out.pov", "v.out.svg", "v.out.vtk", "v.outlier", "v.overlay", "v.pack", "v.parallel", "v.patch", "v.perturb", "v.proj", "v.qcount", "v.random", "v.rast.stats", "v.reclass", "v.rectify", "v.report", "v.sample", "v.segment", "v.select", "v.split", "v.surf.bspline", "v.surf.idw", "v.surf.rst", "v.surf.rst.cvdev", "v.to.3d", "v.to.lines", "v.to.points", "v.to.rast", "v.transform", "v.type", "v.univar", "v.vect.stats", "v.voronoi", "v.voronoi.skeleton", "v.what.rast", "v.what.vect"]

In [ ]:
for idx, tool in enumerate(GRASS_tools):
    print(f"Processing: {idx + 1} / {len(GRASS_tools)}, {t} ")
    url = f"https://grass.osgeo.org/grass84/manuals/{tool}.html"
    print("    URL:", url)
    
    
    print()
    
    if idx > 2:
        break

In [ ]:
soup = get_response(url)
url

In [ ]:
from bs4 import BeautifulSoup
import json

# Sample HTML (In a real case, you would read this from an HTML file)
 

# Parse the HTML content
# soup = BeautifulSoup(html_content, 'html.parser')

# Dictionary to hold headers and associated text content
content_dict = {}

# Function to assemble text under each header, including non-tag inline text
def get_associated_content(header):
    content = []
    next_sibling = header.find_next_sibling()

    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        # Handle elements with tags
        if next_sibling.name in ['a', 'p', 'em', 'div',  'b', 'dd', 'dt', 'dl']:
            content.append(next_sibling.get_text(strip=True))
        # Handle inline text outside of tags
        if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
            content.append(next_sibling.next_sibling.strip())
        next_sibling = next_sibling.find_next_sibling()

    # Join everything together in paragraph form
    return ' '.join(content)

# Extract headers and assemble content for each section
for header in soup.find_all(['h2', 'h3', 'h4']):
    header_text = header.get_text(strip=True)
    paragraph = get_associated_content(header)
    if paragraph:
        content_dict[header_text] = paragraph

# Save the result to a JSON file
with open('assembled_content.json', 'w') as json_file:
    json.dump(content_dict, json_file, indent=4)

print("Content assembled and saved to assembled_content.json")


In [ ]:
from bs4 import BeautifulSoup
import json

soup = get_response(url)

# Dictionary to hold headers and associated paragraphs
content_dict = {}

# Function to assemble text under each header, including non-tag inline text
def get_associated_content(header):
    content = []
    next_sibling = header.find_next_sibling()
    
    # print("next_sibling:", next_sibling)

    # Loop through sibling elements until another header tag is encountered
#     while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
#         # Handle elements with tags
#         if next_sibling and next_sibling.name in ['a', 'p', 'div', 'em', 'b', 'dd', 'dt']:
#             content.append(next_sibling.get_text(strip=True))
#         # Handle inline text outside of tags
#         if next_sibling and isinstance(next_sibling, str):
#             content.append(next_sibling.strip(strip=True))
#         # Move to the next sibling (tag or text node)
#         next_sibling = next_sibling.find_next_sibling()
        
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        # Handle elements with tags
        if next_sibling.name in ['a', 'p', 'em', 'div',  'b', 'dd', 'dt', 'dl']:
            content.append(next_sibling.get_text(strip=True))
        # Handle inline text outside of tags
        if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
            content.append(next_sibling.next_sibling.strip())
        next_sibling = next_sibling.find_next_sibling()


    # Join everything together in paragraph form
    return ' '.join(content)

# Variables to store current hierarchy level
current_h2 = None
current_h3 = None

# Extract headers (h2, h3, h4) and their associated paragraphs
# Iterate through all headers (h2, h3, h4)
for header in soup.find_all(['h2', 'h3', 'h4']):
    header_text = header.get_text(strip=True)
    paragraph = get_associated_content(header)
    
    # print("header.name:", header.name, header_text)
    # print("paragraph:", paragraph)
    
    if header.name == 'h2':
        # Create a new h2 entry
        content_dict[header_text] = {}
        current_h2 = header_text
        current_h3 = None  # Reset h3 context
        content_dict[current_h2] = {}
        content_dict[current_h2]['paragraphs'] = paragraph
        print("header.name:", header.name, current_h2)
        # print("paragraph:", paragraph)

    elif header.name == 'h3' and current_h2:
        # Create a new h3 entry under the current h2
        # content_dict[current_h2][header_text] = {}
        current_h3 = header_text
        # print("content_dict:", content_dict[current_h2])
        print("header.name:", header.name, current_h3)
        content_dict[current_h2][current_h3] = paragraph

#     elif header.name == 'h4' and current_h2 and current_h3:
#         # Create a new h4 entry under the current h3
#         content_dict[current_h2][current_h3][header_text] = paragraph

# for header in soup.find_all(['h2', 'h3', 'h4']):
#     header_text = header.get_text(strip=True)
#     paragraph = get_associated_content(header)
#     if paragraph:
#         content_dict[header_text] = paragraph

        
# Save the result to a JSON file
with open('headers_and_paragraphs.json', 'w') as json_file:
    json.dump(content_dict, json_file, indent=4)

print("Headers and paragraphs extracted and saved to headers_and_paragraphs.json")

In [ ]:
from bs4 import BeautifulSoup
import json


# Function to assemble text under each header, including non-tag inline text
def get_associated_content(header):
    content = []
    next_sibling = header.find_next_sibling()

    # Loop through sibling elements until another header tag (h2, h3, h4) is encountered
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        # Handle elements with tags
        if next_sibling.name in ['a', 'p', 'div', 'em', 'b', 'dd', 'dt']:
            content.append(next_sibling.get_text(strip=True))
        # Handle inline text outside of tags
        if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
            content.append(next_sibling.next_sibling.strip())
        next_sibling = next_sibling.find_next_sibling()

    # Join everything together in paragraph form
    return ' '.join(content)

# Recursive function to process headers and their subheaders
def process_headers(header):
    result = {}
    header_text = header.get_text(strip=True)
    paragraph = get_associated_content(header)

    # Store the paragraph content
    result["content"] = paragraph

    # Recursively process subheaders
    next_sibling = header.find_next_sibling()
    while next_sibling and next_sibling.name != 'h2':  # Stop when encountering the next h2 header
        if next_sibling.name == 'h3':
            result[next_sibling.get_text(strip=True)] = process_headers(next_sibling)
        if next_sibling.name == 'h4':
            result[next_sibling.get_text(strip=True)] = process_headers(next_sibling)
        next_sibling = next_sibling.find_next_sibling()

    return result

# Main dictionary to store all headers and subheaders
content_dict = {}

# Extract and process all h2 headers and recursively process their subheaders
for header in soup.find_all('h2'):
    content_dict[header.get_text(strip=True)] = process_headers(header)

# Save the result to a JSON file
with open('headers_and_subheaders_recursive.json', 'w') as json_file:
    json.dump(content_dict, json_file, indent=4)

print("Headers and subheaders extracted and saved to headers_and_subheaders_recursive.json")


# By header

In [158]:
from bs4 import BeautifulSoup
import json
import copy
import pprint

url = r'https://grass.osgeo.org/grass84/manuals/i.aster.toar.html'

soup = get_response(url)

Title:  i.aster.toar - GRASS GIS manual


In [181]:



# Dictionary to hold headers and their associated content
content_dict = {}


# grass_html = Extract_GRASS_doc(soup)
 

def get_name_content(header, tool_info):
    
    next_sibling = header.find_next_sibling()
    
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        if next_sibling.name in ['a', 'p', 'em', 'div',  'b', 'dd', 'dt', 'dl', 'pre']:
            sibling_text = next_sibling.get_text(strip=True)
            tool_info['name'] = sibling_text
            
        # Handle inline text outside of tags
        if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
            sibling_text = next_sibling.next_sibling.strip()
            tool_info['brief_description'] = sibling_text[2:] # remove the starting "- ". 
            
        next_sibling = next_sibling.find_next_sibling()        
             
    return tool_info

def get_keywords_content(header, tool_info):
    keywords = []
    next_sibling = header.find_next_sibling()
    
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        if next_sibling.name in ['a', 'p', 'em', 'div',  'b', 'dd', 'dt', 'dl', 'pre']:
            sibling_text = next_sibling.get_text(strip=True)            
            keywords.append(sibling_text) 
            
        next_sibling = next_sibling.find_next_sibling()  
    tool_info['keywords'] = keywords
             
    return tool_info

def get_synopsis_content(header, tool_info):
    content_lines = []
    
    # get tool name
    next_sibling = header.find_next_sibling()
    sibling_text = next_sibling.get_text(strip=True)    
    content_lines.append(sibling_text)
    
    # get tool --help
    next_sibling = next_sibling.find_next_sibling()    
    sibling_text = next_sibling.get_text(strip=True)    
    content_lines.append(sibling_text)
    
    
    next_sibling = next_sibling.find_next_sibling() # skip a <br>
    
    # get tool command    
    next_sibling = next_sibling.find_next_sibling()    
    sibling_text = next_sibling.get_text(strip=True)    
    content_lines.append(sibling_text)
     
    tool_info['synopsis'] = content_lines
             
    return tool_info

def get_flags_content(header, tool_info):
    content_dict = {}    
    next_sibling = header.find_next_sibling()  
    dl = next_sibling.find('dl')
    # Iterate over <dt> and <dd> tags
    for dt in next_sibling.find_all('dt'):
        term = dt.get_text(strip=True)  # Get the text in <dt>
        dd = dt.find_next_sibling('dd')  # Get the corresponding <dd>
        if dd:
            description = dd.get_text(strip=True)  # Get the text in <dd>
            content_dict[term] = description
     
    tool_info['Flags'] = content_dict
             
    return tool_info

def get_parameters_content(header, tool_info):
    content_dict = {}    
    next_sibling = header.find_next_sibling()  
    dl = next_sibling.find('dl')
    # Iterate over <dt> and <dd> tags
    for dt in next_sibling.find_all('dt'):
        term = dt.get_text(strip=True)  # Get the text in <dt>
        dd = dt.find_next_sibling('dd')  # Get the corresponding <dd>
        if dd:
            description = dd.get_text(strip=True)  # Get the text in <dd>
            content_dict[term] = description
     
    tool_info['Parameters'] = content_dict
             
    return tool_info

def get_description_content(header, tool_info):
    paragraphs = []
    paragarph = ""
    next_sibling = header.next_sibling
    # print("next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
    # print("next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
    
    while next_sibling and next_sibling.name not in ['h2']:
        if next_sibling.name in ['a', 'em', 'div',  'b', 'dd', 'dt', 'dl', 'pre']:
            # print("next_sibling.name:", next_sibling.name)
            sibling_text = next_sibling.get_text(strip=True)            
            paragarph += sibling_text            
            # print("type of next_sibling:", type(next_sibling))
            # print("sibling_text:", sibling_text)
            
        # if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
        if isinstance(next_sibling, str):
            sibling_text = next_sibling.get_text().replace('\n', '')
            
            # print("      string， sibling_text:", sibling_text)
            
            if next_sibling.next_sibling.name in ["h2", "h3", "h4"]:
                paragraphs.append(sibling_text)
                # print("paragraphs:", paragraphs)
                
            else:
                
                paragarph += sibling_text
            # print("string， sibling_text:", sibling_text)
            # print("type of next_sibling.next_sibling:", type(next_sibling.next_sibling))
            # print(" next_sibling.next_sibling:", next_sibling.next_sibling)
             
            # next_sibling.next_sibling
            # print("next_sibling.name:", next_sibling.name)
            # print("paragraphs:", paragraphs)
            
        if next_sibling.name in ['p']:
            sibling_text = next_sibling.get_text(strip=True)   
            paragraphs.append(paragarph)
            
            # print("paragraphs:", paragraphs)
            # print("p， paragarph:", paragarph)
            paragarph = ""
            paragraphs.append(sibling_text)            
            # print("sibling_text:", sibling_text)
            # print("paragraphs:", paragraphs)
            
             
        if next_sibling.name in ['ul']:
            
            if paragarph:
                paragraphs.append(paragarph)
                # print("paragraphs:", paragraphs)
                paragarph = ""
            
            sibling_text = next_sibling.get_text(strip=False)
            ulist = sibling_text.split('\n')
            
            # print("ulist:", ulist            
            # remove emptry strings
            ulist = [f"•{item}" for item in ulist if item]  # correct
            # print("ulist:", ulist)
            # ulist = [ for item in ulist if item]
            
            # print("paragraphs:", paragraphs)
            
            paragraphs += ulist
            
            # paragraphs.append(sibling_text)
            # print("paragraphs:", paragraphs)
              
        # next_sibling = next_sibling.find_next_sibling()  # cannot find strings that are not enclosed in tags
        next_sibling = next_sibling.next_sibling  # can find strings that are not enclosed in tags
        # print("next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
        
        
    tool_info['description'] = paragraphs
             
    return tool_info
 
    
def get_notes_content(header, tool_info):
    paragraphs = []
    paragarph = ""
    next_sibling = header.next_sibling
    # print("next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
    
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        if next_sibling.name in ['a', 'em',  'b', 'dd', 'dt', 'dl']:
            sibling_text = next_sibling.get_text(strip=True)            
            paragarph += sibling_text             
            
        if isinstance(next_sibling, str):
            sibling_text = next_sibling.get_text().replace('\n', '')
            
            # print("     string next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
            
            if next_sibling.next_sibling.name in ["h2", "h3", "h4"]:
                paragraphs.append(sibling_text)               
            else:                
                paragarph += sibling_text
                
            if next_sibling.next_sibling.name in ['pre', 'h2', 'h3', 'h4']:  # end
                paragraphs.append(paragarph) 
                paragarph = ""
            
        if next_sibling.name in ['p']:
            sibling_text = next_sibling.get_text(strip=True)   
            paragraphs.append(paragarph)            
            paragarph = ""
            paragraphs.append(sibling_text)            
            
        if next_sibling.name in ['div']:
            
            if paragarph:
                paragraphs.append(paragarph)
                # print("paragraphs:", paragraphs)
                paragarph = ""
            
            sibling_text = next_sibling.get_text(strip=False)
            # print("sibling_text:", sibling_text)
 
  
            # print("paragraphs:", paragraphs)
            
  
            paragraphs.append(sibling_text)
            # print("paragraphs:", paragraphs)
              
        # next_sibling = next_sibling.find_next_sibling()  # cannot find strings that are not enclosed in tags
        next_sibling = next_sibling.next_sibling  # can find strings that are not enclosed in tags
        # print("next_sibling:", next_sibling.name, next_sibling.get_text(strip=True))
        
    tool_info['notes'] = paragraphs
             
    return tool_info

def get_see_also_content(header, tool_info):
    links = []
    next_sibling = header.find_next_sibling()
    # print("header:", header, header.name)
    # print("next_sibling:", next_sibling.name, next_sibling,  next_sibling.get_text(strip=True))
    # print("next_sibling:", header.find_next_sibling())
    
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        if next_sibling.name in ['a', 'em']:
            sibling_text = next_sibling.get_text(strip=True)            
            links =   sibling_text.split(",")   
            
                if next_sibling.name in ['a', 'em']:
            sibling_text = next_sibling.get_text(strip=True)            
            links =   sibling_text.split(",")  
            
        next_sibling = next_sibling.find_next_sibling()  # cannot find strings that are not enclosed in tags
        
        # .next_sibling: This method only navigates the immediate next sibling node in the document. The sibling could be another tag, a text node, or even whitespace. If there is any text (including line breaks, spaces, etc.) between the <h2> and <em> tags, .next_sibling will return that text or whitespace instead of the <em> tag.
             
    tool_info['see_also'] = links
             
    return tool_info


tool_info = {}
for header in soup.find_all(['h2', 'h3', 'h4']):
    header_text = header.get_text(strip=True)
    
    print(header_text)     
    
    if header_text == 'NAME':
        get_name_content(header, tool_info)
        
    if header_text == 'KEYWORDS':
        get_keywords_content(header, tool_info)
        
    if header_text == 'SYNOPSIS':
        get_synopsis_content(header, tool_info)
        
    if header_text == 'Flags:':
        get_flags_content(header, tool_info)
        
    if header_text == 'DESCRIPTION':
        get_description_content(header, tool_info)
        
    if header_text == 'NOTES':
        get_notes_content(header, tool_info)
    
    if header_text == 'SEE ALSO':
        get_see_also_content(header, tool_info)
        
#     if header_text == 'AUTHOR':
#         get_author_content(header, tool_info)      
        
#     if header_text == 'SOURCE CODE':
#         get_source_code_content(header, tool_info)              
        
pprint.pprint(tool_info, width=800)

NAME
KEYWORDS
SYNOPSIS
Flags:
Parameters:
Table of contents
DESCRIPTION
NOTES
SEE ALSO
AUTHOR
SOURCE CODE
{'Flags': {'--help': 'Print usage summary', '--overwrite': 'Allow output files to overwrite existing files', '--quiet': 'Quiet module output', '--ui': 'Force launching GUI dialog', '--verbose': 'Verbose module output', '-a': 'VNIR is High Gain', '-b': 'SWIR is High Gain', '-c': 'VNIR is Low Gain 1', '-d': 'SWIR is Low Gain 1', '-e': 'SWIR is Low Gain 2', '-r': 'Output is radiance (W/m2)'},
 'brief_description': 'Calculates Top of Atmosphere Radiance/Reflectance/Brightness Temperature from ASTER DN.',
 'description': ['i.aster.toar calculates the Top Of Atmosphere (TOA) reflectancefor Terra-ASTER L1B in the visible, NIR and SWIR bands (9+1 bands) andbrightness temperature for the TIR bands (5 bands), all from L1B DN values.It is useful to apply after import of original ASTER imagery thatis generally in standard DN values range.', 'The order of input bands is', '• VNIR: 1,2,3N,3B', '

In [179]:
for line in tool_info['see_also']:
    print(line)

i.landsat.toar
r.in.aster


In [162]:
for line in tool_info['notes']:
    print(line)


Internally, a gain code is defined to modify gains according to spectralbands following the GeoSystems GmbH ATCOR Ver. 2.0 Calibration Files.The function is defined in gain_aster.c file.
/*Gain Code*/
    /*0 - High (Not Applicable for band 10-14: TIR)*/
    /*1 - Normal*/
    /*2 - Low 1(Not Applicable for band 10-14: TIR)*/
    /*3 - Low 2(Not Applicable for Band 1-3N/B and 10-14)*/





In [187]:
 requests.get(url).content

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head>\n <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n <meta name="Author" content="GRASS Development Team">\n <meta http-equiv="content-language" content="en-us">\n <meta name="viewport" content="width=device-width, initial-scale=1">\n <title>i.aster.toar - GRASS GIS manual</title>\n <meta name="description" content="i.aster.toar: Calculates Top of Atmosphere Radiance/Reflectance/Brightness Temperature from ASTER DN.">\n <meta name="keywords" content="imagery, radiometric conversion, radiance, reflectance, brightness temperature, satellite, ASTER">\n <link rel="stylesheet" href="grassdocs.css" type="text/css">\n</head>\n<body bgcolor="white">\n<div id="container">\n\n<a href="index.html"><img src="grass_logo.png" alt="GRASS logo"></a>\n<script>\n// Create hamburger menu TOC HTML elements by the JavaScript\nlet temp = document.createElement(\'template\');\nconst toc = \'<ul class="toc-mob

In [195]:
# ! pip install html2text
# ! pip install markdownify

import html2text
from markdownify import markdownify


html_content = """
<h1>Title</h1>
<p>This is a <strong>bold</strong> paragraph.</p>
<a href="https://example.com">Link to Example</a>
"""

# Convert HTML to Markdown
# markdown_content = html2text.html2text( requests.get(url).content)
markdown_content = markdownify( requests.get(url).content)

# Print the result
print(markdown_content)









i.aster.toar \- GRASS GIS manual






[![GRASS logo](grass_logo.png)](index.html)



---


NAME
----


***i.aster.toar*** \- Calculates Top of Atmosphere Radiance/Reflectance/Brightness Temperature from ASTER DN.
KEYWORDS
--------


[imagery](imagery.html), [radiometric conversion](topic_radiometric_conversion.html), [radiance](keywords.html#radiance), [reflectance](keywords.html#reflectance), [brightness temperature](keywords.html#brightness temperature), [satellite](keywords.html#satellite), [ASTER](keywords.html#ASTER)
SYNOPSIS
--------


**i.aster.toar**  

**i.aster.toar \-\-help**  

**i.aster.toar** \[\-**rabcde**] **input**\=*name*\[,*name*,...] **dayofyear**\=*float* **sun\_elevation**\=*float* **output**\=*name* \[\-\-**overwrite**] \[\-\-**help**] \[\-\-**verbose**] \[\-\-**quiet**] \[\-\-**ui**] 


### Flags:



**\-r**
Output is radiance (W/m2\)
**\-a**
VNIR is High Gain
**\-b**
SWIR is High Gain
**\-c**
VNIR is Low Gain 1
**\-d**
SWIR is Low Gain 1
**\-e**
SWIR is

In [161]:
tool_info

{'name': 'i.aster.toar',
 'brief_description': 'Calculates Top of Atmosphere Radiance/Reflectance/Brightness Temperature from ASTER DN.',
 'keywords': ['imagery',
  'radiometric conversion',
  'radiance',
  'reflectance',
  'brightness temperature',
  'satellite',
  'ASTER'],
 'synopsis': ['i.aster.toar',
  'i.aster.toar --help',
  'i.aster.toar[-rabcde]input=name[,name,...]dayofyear=floatsun_elevation=floatoutput=name[--overwrite]  [--help]  [--verbose]  [--quiet]  [--ui]'],
 'Flags': {'-r': 'Output is radiance (W/m2)',
  '-a': 'VNIR is High Gain',
  '-b': 'SWIR is High Gain',
  '-c': 'VNIR is Low Gain 1',
  '-d': 'SWIR is Low Gain 1',
  '-e': 'SWIR is Low Gain 2',
  '--overwrite': 'Allow output files to overwrite existing files',
  '--help': 'Print usage summary',
  '--verbose': 'Verbose module output',
  '--quiet': 'Quiet module output',
  '--ui': 'Force launching GUI dialog'},
 'description': ['i.aster.toar calculates the Top Of Atmosphere (TOA) reflectancefor Terra-ASTER L1B in th

In [160]:
# pprint.pprint(tool_info, width=800)
for line in tool_info['description']:
    print(line)

# len(soup.find_all("p"))
# soup.find_all("p")[0]

i.aster.toar calculates the Top Of Atmosphere (TOA) reflectancefor Terra-ASTER L1B in the visible, NIR and SWIR bands (9+1 bands) andbrightness temperature for the TIR bands (5 bands), all from L1B DN values.It is useful to apply after import of original ASTER imagery thatis generally in standard DN values range.
The order of input bands is
• VNIR: 1,2,3N,3B
• SWIR: 4,5,6,7,8,9
• TIR: 10,11,12,13,14
in one comma-separated list.


In [ ]:


class Extract_GRASS_doc:
    def __init__(self, soup):
        self.soup = soup        
        
    def NAME(self):
        return
    
    def KEYWORDS(self):
        return
    
    def SYNOPSIS(self):
        return
    
    def Flags(self):
        return
    
    def Parameters(self):
        return
    
    def DESCRIPTION(self):
        return
    
    def SEE_ALSO(self):
        return
 
    def NOTES(self):
        return    
    
    def AUTHOR(self):
        return
    
    
    def SOURCE_CODE(self):
        return    
    
    

    
    # grass_html.NAME()

In [ ]:
from bs4 import BeautifulSoup
import json

soup = get_response(url)

# Dictionary to hold headers and their associated content
 
content_dict = {}

# Function to get content associated with each header
def get_associated_content(header):
    content = []
    next_sibling = header.find_next_sibling()
    
    while next_sibling and next_sibling.name not in ['h2', 'h3', 'h4']:
        # Handle elements with tags
        if next_sibling.name in ['a', 'p', 'em', 'div',  'b', 'dd', 'dt', 'dl', 'pre']:
            sibling_text = next_sibling.get_text(strip=True)
            print("sibling_text:", sibling_text)
            content.append(sibling_text)
        # Handle inline text outside of tags
        if next_sibling.next_sibling and isinstance(next_sibling.next_sibling, str):
            sibling_text = next_sibling.next_sibling.strip()
            print("sibling_text:", sibling_text)
            content.append(sibling_text)
            
        next_sibling = next_sibling.find_next_sibling()
    
    return ' '.join(content)

# Variables to track current headers
current_h2 = None
current_h3 = None



# Iterate over all headers (h2, h3, h4) and extract their content
for header in soup.find_all(['h2', 'h3', 'h4']):
    header_text = header.get_text(strip=True)
    content = get_associated_content(header)
    
    if header.name == 'h2':
        # New h2 header, reset the hierarchy
        content_dict[header_text] = {"content": content}
        current_h2 = header_text
        current_h3 = None
    
    elif header.name == 'h3' and current_h2:
        # h3 is nested directly under the current h2
        content_dict[current_h2][header_text] = {"content": content}
        current_h3 = header_text
    
    elif header.name == 'h4' and current_h2 and current_h3:
        # h4 is nested directly under the current h3
        content_dict[current_h2][header_text] = content

# Save the result to a JSON file
with open('headers_and_content.json', 'w') as json_file:
    json.dump(content_dict, json_file, indent=4)

print("Headers and content extracted and saved to headers_and_content.json")

In [ ]:
import pprint as pp
pp.pprint(content_dict )